In [2]:
import sqlite3
import pandas as pd
import random

# === Configuration ===
USER_DB = "users.db"
GYM_DATASET_CSV = "megaGymDataset_trimmed.csv"

# === Initialize Database ===
def init_db():
    conn = sqlite3.connect(USER_DB)
    c = conn.cursor()

    # Create users table
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            gender TEXT,
            major TEXT,
            preferences TEXT, 
            gym TEXT
        )
    ''')

    # Create ratings table
    c.execute('''
        CREATE TABLE IF NOT EXISTS ratings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            exercise TEXT,
            rating INTEGER CHECK(rating BETWEEN 1 AND 5),
            UNIQUE(username, exercise)
        )
    ''')

    conn.commit()
    conn.close()

# === Generate Dummy Users ===
def generate_dummy_users(num_users=20):
    genders = ['Male', 'Female', 'Other', 'Prefer not to say']
    majors = [
        "Engineering/CS", "Math/Science", "Liberal Arts", 
        "Humanities", "Education", "Fine Arts", "Business", 
        "Social Science", "Health", "Other"
    ]
    preferences_samples = [
        "strength", "cardio", "yoga", "crossfit", "HIIT", "pilates", "endurance"
    ]

    conn = sqlite3.connect(USER_DB)
    c = conn.cursor()

    for i in range(1, num_users + 1):
        username = f"user{i}"
        gender = random.choice(genders)
        major = random.choice(majors)
        preferences = ', '.join(random.sample(preferences_samples, k=2))
        gym = {
            "Engineering/CS": "Polo",
            "Math/Science": "Polo",
            "Liberal Arts": "Southside",
            "Humanities": "Southside",
            "Education": "Southside",
            "Fine Arts": "Southside",
            "Business": "Main Rec",
            "Social Science": "Main Rec",
            "Health": "Main Rec",
            "Other": "Main Rec"
        }[major]

        c.execute('''
            INSERT OR IGNORE INTO users (username, gender, major, preferences, gym)
            VALUES (?, ?, ?, ?, ?)
        ''', (username, gender, major, preferences, gym))

    conn.commit()
    conn.close()

# === Generate Dummy Ratings ===
def generate_dummy_ratings(ratings_per_user=10):
    df = pd.read_csv(GYM_DATASET_CSV)
    exercise_titles = df['Title'].dropna().unique().tolist()

    conn = sqlite3.connect(USER_DB)
    c = conn.cursor()

    c.execute("SELECT username FROM users")
    users = [u[0] for u in c.fetchall()]

    for username in users:
        rated_exercises = random.sample(exercise_titles, min(ratings_per_user, len(exercise_titles)))

        for exercise in rated_exercises:
            rating = random.randint(1, 5)
            c.execute('''
                INSERT OR REPLACE INTO ratings (username, exercise, rating)
                VALUES (?, ?, ?)
            ''', (username, exercise, rating))

    conn.commit()
    conn.close()

# === Run the script ===
if __name__ == "__main__":
    print("Initializing database...")
    init_db()

    print("Generating dummy users...")
    generate_dummy_users(num_users=50)

    print("Generating dummy ratings...")
    generate_dummy_ratings(ratings_per_user=10)

    print("Dummy data generation completed!")


Initializing database...
Generating dummy users...
Generating dummy ratings...
Dummy data generation completed!
